In [15]:
import brightway25 as bw
import bw2io as bi 
import bw2data as bd 
import bw2calc as bc 

In [16]:
# to import forwast
import os
import zipfile
from bw2data.utils import download_file

In [4]:
from premise import *

ModuleNotFoundError: No module named 'premise'

## This notebook

The objective of this notebook is to demonstrate how to solve the "My First LCA" exercise from the ESA course using the Brightway and ActivityBrowser tools. Additionally, it provides supplementary information on these tools.

- Author (and contact):
    - Alvaro Hahn (alvaro.hahn-menacho@psi.ch)


### Useful material

1. General info on how to use Brightway:
   - https://docs.brightway.dev/en/latest/content/examples/brightway-examples/data_import/ecoinvent_import.html
   - https://docs.brightway.dev/en/legacy/
3. General info on Activity-Browser:
   - https://github.com/LCA-ActivityBrowser/activity-browser
   - https://www.youtube.com/@activity-browser
4. Interested in prospective LCA 🧐:
   - https://github.com/polca/premise

### 1. Set project and create biosphere

In [24]:
bd.projects.set_current("myfirstLCA") #Creating/accessing the project
bi.bw2setup() #Importing elementary flows, LCIA methods and some other data
bd.databases 

Biosphere database already present!!! No setup is needed


Databases dictionary with 1 object(s):
	biosphere3

## 2. Import forwast database (and ecoinvent)

NameError: name 'BW2Package' is not defined

In [38]:
ei_path = "/Users/hannegoericke/Documents/Ecoinvent/ecoinvent 3.9.1_cutoff_ecoSpold02/datasets"


#ecoinvent
if 'ecoinvent 3.9.1 cutoff' in bd.databases:
    print("Database has already been imported.")
else:
    ei39cut = bi.SingleOutputEcospold2Importer(ei_path, 'ecoinvent 3.9.1 cutoff')
    ei39cut
    ei39cut.apply_strategies()
    ei39cut.statistics()
    ei39cut.write_database()

Extracting XML data from 21238 datasets
Extracted 21238 datasets in 31.43 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy: 

100%|████████████████████████████████████| 21238/21238 [00:31<00:00, 672.79it/s]


Vacuuming database 
Created database: ecoinvent 3.9.1 cutoff


In [44]:
bd.databases

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent 3.9.1 cutoff

## 3. Import Excel database

In [42]:
imp = bi.ExcelImporter(os.path.join("data/LCI_cars.xlsx"))
imp.apply_strategies()
imp.match_database(fields=('name', 'unit', 'location'))
imp.match_database("ecoinvent 3.9.1 cutoff", fields=('name', 'unit', 'location'))
imp.statistics()

Extracted 1 worksheets in 0.05 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 6.81 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
8 datasets
27 exchanges
0 unlinked exchanges
  


(8, 27, 0)

### ⚠️ Warning: It is key to have 0 unlinked exchanges. Otherwise, it will not be possible to solve the system.

#### Only once we have 0 unliked exchanges we write our database. What if we need to troubleshoot? We run:

In [ ]:
# imp.write_excel()

In [45]:
imp.write_database()

100%|██████████████████████████████████████████| 8/8 [00:00<00:00, 10321.26it/s]


Vacuuming database 
Created database: esa_exercise


In [56]:
bd.databases

Databases dictionary with 3 object(s):
	biosphere3
	ecoinvent 3.9.1 cutoff
	esa_exercise

## 4. Exploring the database

In [58]:
wbi = bd.Database("esa_exercise")
for act in wbi:
    print(act)

'power generation' (kilowatt hour, CH, None)
'driving the ICEV' (kilometer, CH, None)
'EV production' (unit, CH, None)
'ICEV production' (unit, CH, None)
'natural gas supply' (cubic meter, CH, None)
'diesel supply' (liter, CH, None)
'driving the EV' (kilometer, CH, None)
'battery manufacturing' (unit, CH, None)


In [59]:
for key in bd.methods:
   if 'climate change' in key:
       print(key)

('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)')
('Ecological Scarcity 2021', 'climate change', 'global warming potential (GWP100)')
('EF v3.0', 'climate change', 'global warming potential (GWP100)')
('EF v3.1', 'climate change', 'global warming potential (GWP100)')
('IMPACT 2002+ (Endpoint)', 'climate change', 'climate change')
('IMPACT 2002+ (Endpoint)', 'climate change', 'total')
('IPCC 2013', 'climate change', 'global temperature change potential (GTP100)')
('IPCC 2013', 'climate change', 'global temperature change potential (GTP20)')
('IPCC 2013', 'climate change', 'global warming potential (GWP100)')
('IPCC 2013', 'climate change', 'global warming potential (GWP20)')
('IPCC 2021', 'climate change', 'global temperature change potential (GTP100)')
('IPCC 2021', 'climate change', 'global temperature change potential (GTP50)')
('IPCC 2021', 'climate change', 'global warming potential (GWP100)')
('IPCC 2021', 'climate change', 'global warming potential (GWP20)'

## 5. LCA

#### Goal and scope

In [60]:
# functional unit (driving 100km with each vehicle alternative)
ev_driving = [act for act in wbi if 'driving the EV' in act['name'] and 'CH' in act['location']][0]
functional_unit_EV = {ev_driving:100}
functional_unit_EV

{'driving the EV' (kilometer, CH, None): 100}

In [61]:
icev_driving = [act for act in wbi if 'driving the ICEV' in act['name'] and 'CH' in act['location']][0]
functional_unit_icev = {icev_driving:100}
functional_unit_icev

{'driving the ICEV' (kilometer, CH, None): 100}

In [64]:
# Method: let's select EF3.0
CC_method = [m for m in bd.methods if 'EF v3.0' in str(m) and  'climate change' in str(m) and 'global warming potential (GWP100)' in str(m)][0]
CC_method

('EF v3.0 no LT',
 'climate change no LT',
 'global warming potential (GWP100) no LT')

#### LCI + LCIA

In [69]:
lca_ev = bc.LCA(functional_unit_EV,CC_method)
lca_ev.lci()
lca_ev.lcia()
lca_ev.score

11.232424827019543

In [70]:
lca_icev = bc.LCA(functional_unit_icev,CC_method)
lca_icev.lci()
lca_icev.lcia()
lca_icev.score

24.750991772952766

#### Interpretation

In [71]:
print('- Driving 100km using an electric car yields an impact of : {:.2f} kgCO2eq'.format(lca_ev.score))
print('- Driving 100km using a diesel car yields an impact of : {:.2f} kgCO2eq'.format(lca_icev.score))
if lca_ev.score < lca_icev.score:
    print('- Choosing an electric vehicle is the better alternative for climate change mitigation, resulting in a {:.2f}% reduction in impact compared to an internal combustion engine vehicle.'
          .format((lca_icev.score - lca_ev.score) / lca_icev.score * 100))
else:
    print('- Choosing an internal combustion engine vehicle is the better alternative for climate change mitigation, resulting in a {:.2f}% reduction in impact compared to an electric vehicle.'
          .format((lca_ev.score - lca_icev.score) / lca_ev.score * 100))

- Driving 100km using an electric car yields an impact of : 11.23 kgCO2eq
- Driving 100km using a diesel car yields an impact of : 24.75 kgCO2eq
- Choosing an electric vehicle is the better alternative for climate change mitigation, resulting in a 54.62% reduction in impact compared to an internal combustion engine vehicle.


## 6. Prospective LCA

Here comes premise

Upon the database extraction, you can import some of your Brightway2-compatible inventories like so:

In [72]:
clear_cache()

NameError: name 'clear_cache' is not defined

In [73]:
ndb = NewDatabase(
            scenarios=[
                {"model":"remind", "pathway":"SSP2-Base", "year":2050},
                {"model":"remind", "pathway":"SSP2-PkBudg500", "year":2050},
            ],
            source_db="ecoinvent 3.9.1 cutoff", 
            source_version="3.9.1",
            key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
            additional_inventories= [ # <-- this is NEW
                {"filepath": r"C:\Users\hahnme_a\Desktop\ETH\Teaching LCA\Anna\pLCA_test.xlsx", "ecoinvent version": "3.9.1"}, # <-- this is NEW
            ] # <-- this is NEW
                 )

NameError: name 'NewDatabase' is not defined

In [7]:
ndb.update_all()

`update_all()` will skip the following steps:
update_two_wheelers(), update_cars(), and update_buses()
If you want to update these steps, please run them separately afterwards.
Done!



In [8]:
ndb.write_db_to_brightway()

Write new database(s) to Brightway.
One or multiple duplicates detected. Removing them...
Database ecoinvent_cutoff_3.9_remind_SSP2-Base_2050 already exists: it will be overwritten.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:13


Title: Writing activities to SQLite3 database:
  Started: 10/24/2023 14:30:21
  Finished: 10/24/2023 14:31:35
  Total time elapsed: 00:01:13
  CPU %: 99.10
  Memory %: 31.42
Created database: ecoinvent_cutoff_3.9_remind_SSP2-Base_2050
Database ecoinvent_cutoff_3.9_remind_SSP2-PkBudg500_2050 already exists: it will be overwritten.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:57


Title: Writing activities to SQLite3 database:
  Started: 10/24/2023 14:34:15
  Finished: 10/24/2023 14:35:13
  Total time elapsed: 00:00:57
  CPU %: 99.40
  Memory %: 30.21
Created database: ecoinvent_cutoff_3.9_remind_SSP2-PkBudg500_2050
Generate scenario report.
Report saved under C:\Users\hahnme_a\Desktop\ETH\Teaching LCA\Anna\export\scenario_report.
Generate change report.
Report saved under C:\Users\hahnme_a\Desktop\ETH\Teaching LCA\Anna.


## LCIA